In [234]:
import random
import rospy
import time
from turtlesim.srv import Spawn
from turtlesim.srv import Kill
from turtlesim.srv import SetPen
from turtlesim.srv import TeleportAbsolute
from geometry_msgs.msg import Twist
from std_msgs.msg import String
rospy.init_node("serwis_node_Leonardo")
g_list_pol=["none","none","none","none"]
global g_list_pol
def ustaw_pol(data):
    #g_list_pol=String()
    g_list_pol[0]=data.data
def ustaw_x(data):
    #g_list_pol=String()
    g_list_pol[1]=data.data
def ustaw_y(data):
    #g_list_pol=String()
    g_list_pol[2]=data.data
def ustaw_zad(data):
    #g_list_pol=String()
    g_list_pol[3]=data.data
        
class Robot:
    
     
    def __init__(self, name, x, y, theta,r,g,b):
        # inicjalizacja klasy
        self.name = name
        self.x = x
        self.y = y
        self.r = r
        self.g = g
        self.b = b
        self.theta=theta
        self.dok_x=x
        self.dok_y=y
        self.battery=0.99
        self.dok_theta=theta
        self.akt_pol="none"
        self.akt_zad="none"
        create = rospy.ServiceProxy('spawn', Spawn)
        create(self.dok_x, self.dok_y, self.dok_theta, name)
        pen = rospy.ServiceProxy(name+'/set_pen', SetPen)
        pen(r, g, b, 5,0)
        
        
    def sepuku(self):
        kill = rospy.ServiceProxy('kill', Kill)
        kill(self.name)
        
    def rezurektrion(self):
        create = rospy.ServiceProxy('spawn', Spawn)
        create(self.dok_x, self.dok_y, self.dok_theta, self.name)
        
    def pose(self):
        print("{:f}, {:f}, {:f}".format(self.x, self.y, self.theta))
    
    def pen_on(self):
            pen = rospy.ServiceProxy(self.name+'/set_pen', SetPen)
            pen(self.r, self.g, self.b, 5,0)
    def pen_off(self):
            pen = rospy.ServiceProxy(self.name+'/set_pen', SetPen)
            pen(self.r, self.g, self.b, 5,1)
    
    def obr(self, kier):
        if kier=='p':
            if self.theta==4.71:
                theta=1.57
            else:
                theta=0-self.theta
            self.theta=0
        elif kier=='g':
            theta=1.57-self.theta
            self.theta=1.57
        elif kier=='l':
            theta=3.14-self.theta
            self.theta=3.14
        elif kier=='d':
            if self.theta==0:
                theta=-1.57
            else:
                theta=4.71-self.theta
            self.theta=4.71
        pub_speed=rospy.Publisher(self.name+"/cmd_vel",Twist,queue_size=10)
        msg = Twist()
        msg.linear.x = 0
        msg.angular.z = theta
        pub_speed.publish(msg)
        self.rozladuj()
        rospy.sleep(1)
    
        
    def move(self, odl):
        if self.theta==0:
            self.x+=odl
        elif self.theta==1.57:
            self.y+=odl
        elif self.theta==3.14:
            self.x-=odl
        elif self.theta==4.71:
            self.y-=odl
        pub_speed=rospy.Publisher(self.name+"/cmd_vel",Twist,queue_size=10)
        msg = Twist()
        msg.linear.x = odl
        pub_speed.publish(msg)
        self.rozladuj()
        rospy.sleep(1)
      
    def go(self,x,y):
        if self.y==10.5:
            self.obr('d')
            self.move(1.5)
        if self.x<5.5:
            self.obr('p')
        elif self.x>5.5:
            self.obr('l')
        while self.x!=5.5:
            self.move(0.5)
            #yself.pose()
        if self.y<y:
            self.obr('g')
        elif self.y>y:
            self.obr('d')
        while self.y!=y:
            self.move(0.5)
            self.pose()
            if y>9 and self.y==9:
                break
        if self.x<x:
            self.obr('p')
        elif self.x>x:
            self.obr('l')
        while self.x!=x:
            self.move(0.5)
            self.pose()
        if y>9:
            self.obr('g')
            while self.y!=y:
                self.move(0.5)
    def pracuj(self):
        self.rozladuj()
        self.rozladuj()
        self.rozladuj()
        if self.theta==0:
            Leonardo.obr('g')
           
            Leonardo.obr('d')
            Leonardo.obr('p')
        if self.theta==3.14:
            Leonardo.obr('d')
           
            Leonardo.obr('g')
            Leonardo.obr('l')
    def laduj(self):
        if self.battery<1:
            self.battery+= random.randint(0,9)/1000
                           
    def rozladuj(self):
        if self.battery<1:
            self.battery-= random.randint(0,9)/1000
                           
    def stan_baterii(self):
            print("Bateria: {:.0f}%".format(self.battery*100))
      
    def sprawdz(self,g_list_pol):
       
        if self.akt_zad!=g_list_pol[3] or g_list_pol[0]!=self.akt_pol:
            self.akt_zad=g_list_pol[3]
            self.akt_pol=g_list_pol[0]
            if g_list_pol[1]!="none" and g_list_pol[2]!="none":
                self.go(float(g_list_pol[1]),float(g_list_pol[2]))     
                print("SEX")
            if g_list_pol[3]!="none":
                self.pracuj()
                
        else:
            if self.dok_x!=self.x or self.dok_y!=self.y:
                self.go(self.dok_x,self.dok_y)
                self.obr("d")
                dok = rospy.ServiceProxy(self.name+'/teleport_absolute', TeleportAbsolute)
                dok(self.dok_x, self.dok_y, 4.71)
                #self.akt_zad="charging"
           
            
            
            while self.battery<1:
                self.laduj();
               
                if g_list_pol[0]!=self.akt_pol or g_list_pol[3]!=self.akt_zad :
    
                    break
    def subs_on(self):
        my_sube_pol=rospy.Subscriber(self.name+"/polecenie",String, ustaw_pol)
        my_sube_x=rospy.Subscriber(self.name+"/pol_x",String, ustaw_x)
        my_sube_y=rospy.Subscriber(self.name+"/pol_y",String, ustaw_y)
        my_sube_zad=rospy.Subscriber(self.name+"/pol_zad",String, ustaw_zad)
    def rysuj_dok(self):
            dok = rospy.ServiceProxy(self.name+'/teleport_absolute', TeleportAbsolute)
            self.pen_off()
            dok(self.dok_x-0.5, self.dok_y-0.5, 0)
            self.pen_on()
            dok(self.dok_x+0.5, self.dok_y-0.5, 0)
            dok(self.dok_x+0.5, self.dok_y+0.5, 0)
            dok(self.dok_x-0.5, self.dok_y+0.5, 0)
            dok(self.dok_x-0.5, self.dok_y-0.5, 0)
            self.pen_off()
            dok(self.dok_x, self.dok_y, 4.71)
            #self.pen_on()
       
        


 



In [235]:
Leonardo=Robot("Leonardo",4.0,10.5,4.71, 0,0,255)
Leonardo.rysuj_dok()
Leonardo.subs_on()


In [ ]:
while 1:
    
    print(g_list_pol)
    print(Leonardo.akt_pol +" "+str(Leonardo.x)  +" "+str(Leonardo.y) +" "+Leonardo.akt_zad)
    Leonardo.sprawdz(g_list_pol)
    time.sleep(0.5)

['none', 'none', 'none', 'none']
none 4.0 10.5 none
['none', 'none', 'none', 'none']
none 4.0 10.5 none
['none', 'none', 'none', 'none']
none 4.0 10.5 none
['none', 'none', 'none', 'none']
none 4.0 10.5 none
['none', 'none', 'none', 'none']
none 4.0 10.5 none
['none', 'none', 'none', 'none']
none 4.0 10.5 none
['none', 'none', 'none', 'none']
none 4.0 10.5 none
['none', 'none', 'none', 'none']
none 4.0 10.5 none
['none', 'none', 'none', 'none']
none 4.0 10.5 none
['none', 'none', 'none', 'none']
none 4.0 10.5 none
['none', 'none', 'none', 'none']
none 4.0 10.5 none
['none', 'none', 'none', 'none']
none 4.0 10.5 none
['none', 'none', 'none', 'none']
none 4.0 10.5 none
['none', 'none', 'none', 'none']
none 4.0 10.5 none
['none', 'none', 'none', 'none']
none 4.0 10.5 none
['none', 'none', 'none', 'none']
none 4.0 10.5 none
['none', 'none', 'none', 'none']
none 4.0 10.5 none
['none', 'none', 'none', 'none']
none 4.0 10.5 none
['none', 'none', 'none', 'none']
none 4.0 10.5 none
['none', 'no

['stolik1', '5.0', '8.0', 'none']
stolik1 4.0 10.5 none
['stolik1', '5.0', '8.0', 'none']
stolik1 4.0 10.5 none
['stolik1', '5.0', '8.0', 'none']
stolik1 4.0 10.5 none
['stolik1', '5.0', '8.0', 'none']
stolik1 4.0 10.5 none
['stolik1', '5.0', '8.0', 'none']
stolik1 4.0 10.5 none
['stolik1', '5.0', '8.0', 'none']
stolik1 4.0 10.5 none
['stolik1', '5.0', '8.0', 'none']
stolik1 4.0 10.5 none
['stolik1', '5.0', '8.0', 'none']
stolik1 4.0 10.5 none
['stolik1', '5.0', '8.0', 'none']
stolik1 4.0 10.5 none
['stolik1', '5.0', '8.0', 'none']
stolik1 4.0 10.5 none
['stolik1', '5.0', '8.0', 'none']
stolik1 4.0 10.5 none
['stolik1', '5.0', '8.0', 'none']
stolik1 4.0 10.5 none
['stolik1', '5.0', '8.0', 'none']
stolik1 4.0 10.5 none
['stolik1', '5.0', '8.0', 'none']
stolik1 4.0 10.5 none
['stolik1', '5.0', '8.0', 'none']
stolik1 4.0 10.5 none
['stolik1', '5.0', '8.0', 'none']
stolik1 4.0 10.5 none
['stolik1', '5.0', '8.0', 'none']
stolik1 4.0 10.5 none
['stolik1', '5.0', '8.0', 'none']
stolik1 4.0 10

['stolik1', '5.0', '8.0', 'none']
stolik1 4.0 10.5 none
['stolik1', '5.0', '8.0', 'none']
stolik1 4.0 10.5 none
['stolik1', '5.0', '8.0', 'none']
stolik1 4.0 10.5 none
['stolik1', '5.0', '8.0', 'none']
stolik1 4.0 10.5 none
['stolik1', '5.0', '8.0', 'none']
stolik1 4.0 10.5 none
['stolik1', '5.0', '8.0', 'none']
stolik1 4.0 10.5 none
['stolik1', '5.0', '8.0', 'none']
stolik1 4.0 10.5 none
['stolik1', '5.0', '8.0', 'none']
stolik1 4.0 10.5 none
['stolik1', '5.0', '8.0', 'none']
stolik1 4.0 10.5 none
['stolik1', '5.0', '8.0', 'none']
stolik1 4.0 10.5 none
['stolik1', '5.0', '8.0', 'none']
stolik1 4.0 10.5 none
['stolik1', '5.0', '8.0', 'none']
stolik1 4.0 10.5 none
['stolik1', '5.0', '8.0', 'none']
stolik1 4.0 10.5 none
['stolik1', '5.0', '8.0', 'none']
stolik1 4.0 10.5 none
['stolik1', '5.0', '8.0', 'none']
stolik1 4.0 10.5 none
['stolik1', '5.0', '8.0', 'none']
stolik1 4.0 10.5 none
['stolik1', '5.0', '8.0', 'none']
stolik1 4.0 10.5 none
['stolik1', '5.0', '8.0', 'none']
stolik1 4.0 10

['stolik1', '5.0', '8.0', 'none']
stolik1 4.0 10.5 none
['stolik1', '5.0', '8.0', 'none']
stolik1 4.0 10.5 none
['stolik1', '5.0', '8.0', 'none']
stolik1 4.0 10.5 none
['stolik1', '5.0', '8.0', 'none']
stolik1 4.0 10.5 none
['stolik1', '5.0', '8.0', 'none']
stolik1 4.0 10.5 none
['stolik1', '5.0', '8.0', 'none']
stolik1 4.0 10.5 none
['stolik1', '5.0', '8.0', 'none']
stolik1 4.0 10.5 none
['stolik1', '5.0', '8.0', 'none']
stolik1 4.0 10.5 none
['stolik1', '5.0', '8.0', 'none']
stolik1 4.0 10.5 none
['stolik1', '5.0', '8.0', 'none']
stolik1 4.0 10.5 none
['stolik1', '5.0', '8.0', 'none']
stolik1 4.0 10.5 none
['stolik1', '5.0', '8.0', 'none']
stolik1 4.0 10.5 none
['stolik1', '5.0', '8.0', 'none']
stolik1 4.0 10.5 none
['stolik1', '5.0', '8.0', 'none']
stolik1 4.0 10.5 none
['stolik1', '5.0', '8.0', 'none']
stolik1 4.0 10.5 none
['stolik1', '5.0', '8.0', 'none']
stolik1 4.0 10.5 none
['stolik1', '5.0', '8.0', 'none']
stolik1 4.0 10.5 none
['stolik1', '5.0', '8.0', 'none']
stolik1 4.0 10

['stolik1', '5.0', '8.0', 'none']
stolik1 4.0 10.5 none
['stolik1', '5.0', '8.0', 'none']
stolik1 4.0 10.5 none
['stolik1', '5.0', '8.0', 'none']
stolik1 4.0 10.5 none
['stolik1', '5.0', '8.0', 'none']
stolik1 4.0 10.5 none
['stolik1', '5.0', '8.0', 'none']
stolik1 4.0 10.5 none
['stolik1', '5.0', '8.0', 'none']
stolik1 4.0 10.5 none
['stolik1', '5.0', '8.0', 'none']
stolik1 4.0 10.5 none
['stolik1', '5.0', '8.0', 'none']
stolik1 4.0 10.5 none
['stolik1', '5.0', '8.0', 'none']
stolik1 4.0 10.5 none
['stolik1', '5.0', '8.0', 'none']
stolik1 4.0 10.5 none
['stolik1', '5.0', '8.0', 'none']
stolik1 4.0 10.5 none
['stolik1', '5.0', '8.0', 'none']
stolik1 4.0 10.5 none
['stolik1', '5.0', '8.0', 'none']
stolik1 4.0 10.5 none
['stolik1', '5.0', '8.0', 'none']
stolik1 4.0 10.5 none
['stolik1', '5.0', '8.0', 'none']
stolik1 4.0 10.5 none
['stolik1', '5.0', '8.0', 'none']
stolik1 4.0 10.5 none
['stolik1', '5.0', '8.0', 'none']
stolik1 4.0 10.5 none
['stolik1', '5.0', '8.0', 'none']
stolik1 4.0 10

In [26]:
Leonardo.akt_pol

data: "KURWA"

In [3]:
Leonardo.pose()

4.000000, 10.500000, 4.710000


In [4]:
Leonardo.sepuku()

In [5]:
Leonardo.rezurektrion()

In [48]:

Leonardo.go(8,4)
Leonardo.pracuj()


6.500000, 4.000000, 0.000000
7.000000, 4.000000, 0.000000
7.500000, 4.000000, 0.000000
8.000000, 4.000000, 0.000000


In [58]:
Leonardo.obr('l')
Leonardo.move(3)

In [56]:
Leonardo.obr('g')

In [57]:
Leonardo.obr('p')

In [55]:
Leonardo.obr('d')

In [60]:
!rosservice call reset